# SpaCy

#### Wyświetlenie obrazów zostało zakomentowane, gdyż zatrzymuje kompilację następnych elementów kodu

##### Zadania zostały wykonane na podstawie instruktażu ze strony:
##### https://realpython.com/natural-language-processing-spacy-python/
## Importy

In [66]:
import spacy
import pl_core_news_sm
import re
from spacy.tokenizer import Tokenizer
from collections import Counter
from spacy import displacy
from spacy.matcher import Matcher
import textacy
import warnings

warnings.filterwarnings('ignore')

## Załadowanie polskiego modelu instancji

In [67]:
nlp = pl_core_news_sm.load()

## Utworzenie pliku Doc ze Stringa

In [68]:
introduction_text = ('To jest naturalne przetwarzanie języka w Spacy.')
introduction_doc = nlp(introduction_text)
print([token.text for token in introduction_doc])

['To', 'jest', 'naturalne', 'przetwarzanie', 'języka', 'w', 'Spacy', '.']


## Utworzenie pliku Doc z pliku tekstowego

In [69]:
introduction_file_text = open("text.txt").read()
introduction_file_doc = nlp(introduction_file_text)
print([token.text for token in introduction_file_doc])

['Hello', 'my', 'name', 'is', 'Rafal']


## Detekcja zdań
### Dzieje się to poprzez rozpoznanie początka oraz końca zdań w ciągu znaków

In [70]:
about_text=('Mój generator nie jest pierwszym tego typu ale ma dla mnie ważną zaletę: posługuje się zmielonymi tekstami współczesnymi, a nie np. Panem Tadeuszem, co próbowano wcześniej. Dodatkowo można wybrać sobie jeden z kilku stylów, żeby jak najlepiej wpasować się w pożądany charakter tworzonej strony: do wyboru są m.in. teksty polityczne, naukowe a także przyrodnicze.')
about_doc = nlp(about_text)
sentences = list(about_doc.sents)
print(len(sentences))
for sentence in sentences:
    print(sentence)

4
Mój generator nie jest pierwszym tego typu ale ma dla mnie ważną zaletę: posługuje się zmielonymi tekstami współczesnymi, a nie np.
Panem Tadeuszem, co próbowano wcześniej.
Dodatkowo można wybrać sobie jeden z kilku stylów, żeby jak najlepiej wpasować się w pożądany charakter tworzonej strony: do wyboru są m.in.
teksty polityczne, naukowe a także przyrodnicze.


#### Metoda rozpoznaje koniec zdania po kropce przez co zdania zawierające skróty słowne tworzą błędy.

## Utworzenie własnego ogranicznika

In [71]:
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text == '...' or token.text == 'np.' or token.text == 'm.in.':
            doc[token.i+1].is_sent_start = True
    return doc

ellipsis_text = ('Nie rozumiem, ... dobra. Czy potrafisz np. liczyć? Liczę na twoją pomoc m.in. w matematycę')
custom_nlp = pl_core_news_sm.load()
custom_nlp.add_pipe(set_custom_boundaries, before='parser')
custom_ellipsis_doc = custom_nlp(ellipsis_text)
custom_ellipsis_sentences = list(custom_ellipsis_doc.sents)

for sentence in custom_ellipsis_sentences:
    print(sentence)

ellipsis_doc = nlp(ellipsis_text)
ellipsis_sentences = list(ellipsis_doc.sents)
for sentence in ellipsis_sentences:
    print(sentence)

Nie rozumiem, ...
dobra.
Czy potrafisz np.
liczyć?
Liczę na twoją pomoc m.in.
w matematycę
Nie rozumiem, ... dobra.
Czy potrafisz np.
liczyć?
Liczę na twoją pomoc m.in.
w matematycę


## Tokenizacja

#### Token to podstawowa jednostka tekstu, tokenizacja to rozbicie tekstu na pojedyncze tokeny

### Wypisanie wszystkich kolenów obiektu Doc

In [72]:
for token in about_doc:
    print (token, token.idx)


Mój 0
generator 4
nie 14
jest 18
pierwszym 23
tego 33
typu 38
ale 43
ma 47
dla 50
mnie 54
ważną 59
zaletę 65
: 71
posługuje 73
się 83
zmielonymi 87
tekstami 98
współczesnymi 107
, 120
a 122
nie 124
np 128
. 130
Panem 132
Tadeuszem 138
, 147
co 149
próbowano 152
wcześniej 162
. 171
Dodatkowo 173
można 183
wybrać 189
sobie 196
jeden 202
z 208
kilku 210
stylów 216
, 222
żeby 224
jak 229
najlepiej 233
wpasować 243
się 252
w 256
pożądany 258
charakter 267
tworzonej 277
strony 287
: 293
do 295
wyboru 298
są 305
m.in 308
. 312
teksty 314
polityczne 321
, 331
naukowe 333
a 341
także 343
przyrodnicze 349
. 361


## Wszystkie atrybuty tokena

In [73]:
for token in about_doc:
    print (token, token.idx, token.text_with_ws,token.is_alpha, token.is_punct, token.is_space, token.shape_, token.is_stop)


Mój 0 Mój  True False False Xxx True
generator 4 generator  True False False xxxx False
nie 14 nie  True False False xxx True
jest 18 jest  True False False xxxx True
pierwszym 23 pierwszym  True False False xxxx False
tego 33 tego  True False False xxxx True
typu 38 typu  True False False xxxx False
ale 43 ale  True False False xxx True
ma 47 ma  True False False xx True
dla 50 dla  True False False xxx True
mnie 54 mnie  True False False xxxx True
ważną 59 ważną  True False False xxxx False
zaletę 65 zaletę True False False xxxx False
: 71 :  False True False : False
posługuje 73 posługuje  True False False xxxx False
się 83 się  True False False xxx True
zmielonymi 87 zmielonymi  True False False xxxx False
tekstami 98 tekstami  True False False xxxx False
współczesnymi 107 współczesnymi True False False xxxx False
, 120 ,  False True False , False
a 122 a  True False False x True
nie 124 nie  True False False xxx True
np 128 np True False False xx False
. 130 .  False True False . 

## Utworzenie własnego tokenu

In [74]:
custom_nlp= pl_core_news_sm.load()
prefix_re = spacy.util.compile_prefix_regex(custom_nlp.Defaults.prefixes)
suffix_re = spacy.util.compile_suffix_regex(custom_nlp.Defaults.suffixes)
infix_re = re.compile(r'''[-~]''')
def customize_tokenizer(nlp):
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,suffix_search=suffix_re.search,infix_finditer=infix_re.finditer,token_match=None)
custom_nlp.tokenizer = customize_tokenizer(custom_nlp)
custom_tokenizer_about_doc = custom_nlp(about_text)
print([token.text for token in custom_tokenizer_about_doc])

['Mój', 'generator', 'nie', 'jest', 'pierwszym', 'tego', 'typu', 'ale', 'ma', 'dla', 'mnie', 'ważną', 'zaletę', ':', 'posługuje', 'się', 'zmielonymi', 'tekstami', 'współczesnymi', ',', 'a', 'nie', 'np', '.', 'Panem', 'Tadeuszem', ',', 'co', 'próbowano', 'wcześniej', '.', 'Dodatkowo', 'można', 'wybrać', 'sobie', 'jeden', 'z', 'kilku', 'stylów', ',', 'żeby', 'jak', 'najlepiej', 'wpasować', 'się', 'w', 'pożądany', 'charakter', 'tworzonej', 'strony', ':', 'do', 'wyboru', 'są', 'm.in', '.', 'teksty', 'polityczne', ',', 'naukowe', 'a', 'także', 'przyrodnicze', '.']


## Stop lista

### Są to słowa odrzucane przez przeglądarkę, z powodu ich częstości, w celu zredukowania wielkości zbiorów

In [75]:
spacy_stopwords = spacy.lang.pl.stop_words.STOP_WORDS
len(spacy_stopwords)
for stop_word in list(spacy_stopwords)[:10]:
    print(stop_word)

były
ku
mi
po
nas
wam
iv
niech
acz
natychmiast


### Usunięcie słów z tekstu

In [76]:
for token in about_doc:
    if not token.is_stop:
        print(token)

generator
pierwszym
typu
ważną
zaletę
:
posługuje
zmielonymi
tekstami
współczesnymi
,
np
.
Panem
Tadeuszem
,
próbowano
wcześniej
.
Dodatkowo
wybrać
stylów
,
najlepiej
wpasować
pożądany
charakter
tworzonej
strony
:
wyboru
m.in
.
teksty
polityczne
,
naukowe
przyrodnicze
.


## Lista tokenów nie posiadająca stop słów

In [77]:
about_no_stopword_doc = [token for token in about_doc if not token.is_stop]
print (about_no_stopword_doc)

[generator, pierwszym, typu, ważną, zaletę, :, posługuje, zmielonymi, tekstami, współczesnymi, ,, np, ., Panem, Tadeuszem, ,, próbowano, wcześniej, ., Dodatkowo, wybrać, stylów, ,, najlepiej, wpasować, pożądany, charakter, tworzonej, strony, :, wyboru, m.in, ., teksty, polityczne, ,, naukowe, przyrodnicze, .]



## Lematyzacja

### Jest to redukcja końcówek słów, aby sprowadzić słowo do formy podstawowej

In [78]:
for token in about_doc:
    print(token, token.lemma_)

Mój mój
generator generator
nie nie
jest być
pierwszym pierwszy
tego ten
typu typ
ale ale
ma mieć
dla dla
mnie ja
ważną ważny
zaletę zaleta
: :
posługuje posługiwać
się się
zmielonymi zmielonymi
tekstami tekst
współczesnymi współczesny
, ,
a a
nie nie
np np
. .
Panem pan
Tadeuszem Tadeusz
, ,
co co
próbowano próbować
wcześniej wcześnie
. .
Dodatkowo dodatkowo
można można
wybrać wybrać
sobie sobie
jeden jeden
z z
kilku kilka
stylów styl
, ,
żeby żeby
jak jak
najlepiej dobrze
wpasować wpasować
się się
w w
pożądany pożądany
charakter charakter
tworzonej tworzonej
strony strona
: :
do do
wyboru wybór
są być
m.in m.in
. .
teksty tekst
polityczne polityczny
, ,
naukowe naukowy
a a
także także
przyrodnicze przyrodniczy
. .


## Częstotliwość słów

In [79]:
complete_text =('Starania o zrównoważony rozwój można sprowadzić do postulatu sprawiedliwości międzypokoleniowej. Skoro my możemy korzystać z przyrody taką, jaka jest dziś, to samo prawo powinno przysługiwać naszym dzieciom, wnukom, wnuczkom, ich dzieciom i tak dalej. Ergo nasz rozwój nie może odbywać się kosztem przyrody. Termin ‘zrównoważony’ trudno uznać w tej sytuacji za intuicyjnie zrozumiały, ale ktoś tak wymyślił kilkanaście lat temu i teraz trzeba się go trzymać nie chcąc popaść w lingwistyczno-definicyjne dywagacje. Które oczywiście świetnie nadają się na kolejny tekst. Ale ten jest o czymś innym.')
complete_doc = nlp(complete_text)
words = [token.text for token in complete_doc if not token.is_stop and not token.is_punct]
word_freq = Counter(words)

#### Wyświetlenie najczęściej używanych słów

##### Analizując najczęściej używane słowa, z poza stop listy, można wydedukować temat tekstu

In [80]:
common_words = word_freq.most_common(4)
print (common_words)

[('zrównoważony', 2), ('rozwój', 2), ('przyrody', 2), ('dzieciom', 2)]


#### Wyświetlenie najrzadziej używanych słów

In [81]:
unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
print (unique_words)

['Starania', 'sprowadzić', 'postulatu', 'sprawiedliwości', 'międzypokoleniowej', 'Skoro', 'możemy', 'korzystać', 'taką', 'jaka', 'samo', 'prawo', 'przysługiwać', 'naszym', 'wnukom', 'wnuczkom', 'dalej', 'Ergo', 'odbywać', 'kosztem', 'Termin', 'trudno', 'uznać', 'sytuacji', 'intuicyjnie', 'zrozumiały', 'wymyślił', 'kilkanaście', 'lat', 'trzymać', 'chcąc', 'popaść', 'lingwistyczno', 'definicyjne', 'dywagacje', 'oczywiście', 'świetnie', 'nadają', 'kolejny', 'tekst', 'czymś', 'innym']


## Rozpoznawanie części mowy

In [82]:
for token in about_doc:
    print (token, token.tag_, token.pos_, spacy.explain(token.tag_))


Mój ADJ ADJ adjective
generator SUBST NOUN None
nie QUB PART None
jest FIN VERB None
pierwszym ADJ ADJ adjective
tego ADJ ADJ adjective
typu SUBST NOUN None
ale CONJ CCONJ conjunction
ma FIN VERB None
dla PREP ADP None
mnie PPRON12 PRON None
ważną ADJ ADJ adjective
zaletę SUBST NOUN None
: INTERP PUNCT None
posługuje FIN VERB None
się QUB PART None
zmielonymi ADJ ADJ adjective
tekstami SUBST NOUN None
współczesnymi ADJ ADJ adjective
, INTERP PUNCT None
a CONJ CCONJ conjunction
nie QUB PART None
np BREV X None
. INTERP PUNCT None
Panem SUBST NOUN None
Tadeuszem SUBST NOUN None
, INTERP PUNCT None
co SUBST NOUN None
próbowano IMPS VERB None
wcześniej ADV ADV adverb
. INTERP PUNCT None
Dodatkowo ADV ADV adverb
można PRED VERB None
wybrać INF VERB None
sobie SIEBIE PRON None
jeden ADJ ADJ adjective
z PREP ADP None
kilku NUM NUM numeral
stylów SUBST NOUN None
, INTERP PUNCT None
żeby COMP SCONJ None
jak ADV ADV adverb
najlepiej ADV ADV adverb
wpasować INF VERB None
się QUB PART None
w PREP 

## Wyfiltrowanie słów należących do danych części mowy

In [83]:
nouns = []
adjectives = []
for token in about_doc:
    if token.pos_ == 'NOUN':
        nouns.append(token)
    if token.pos_ == 'ADJ':
        adjectives.append(token)

print(nouns)
print(adjectives)


[generator, typu, zaletę, tekstami, Panem, Tadeuszem, co, stylów, charakter, strony, wyboru, teksty]
[Mój, pierwszym, tego, ważną, zmielonymi, współczesnymi, jeden, pożądany, tworzonej, polityczne, naukowe, przyrodnicze]


## Wizualizacja z displayCy

In [84]:
about_interest_text = ('Różne osoby posiadają autentyczne interesy na ogólne tematy')
about_interest_doc = nlp(about_interest_text)
#displacy.serve(about_interest_doc, style='dep')

## Operacje przygotowujące

#### Odfiltrowanie ważnych tokenów
#### Przeformatowanie tokenów do słów pisanych małą literą oraz zredukowanie do formy podstawowej

In [85]:
def is_token_allowed(token):
    if not token or not token.string.strip() or token.is_stop or token.is_punct:
        return False
    return True

def preprocess_token(token):
    return token.lemma_.strip().lower()

complete_filtered_tokens = [preprocess_token(token)for token in complete_doc if is_token_allowed(token)]
print(complete_filtered_tokens)

['staranie', 'zrównoważony', 'rozwój', 'sprowadzić', 'postulat', 'sprawiedliwość', 'międzypokoleniowy', 'skoro', 'móc', 'korzystać', 'przyroda', 'taki', 'jaki', 'sam', 'prawo', 'przysługiwać', 'nasz', 'dziecko', 'wnuki', 'wnuczka', 'dziecko', 'daleko', 'ergo', 'rozwój', 'odbywać', 'koszt', 'przyroda', 'termin', 'zrównoważony', 'trudno', 'uznać', 'sytuacja', 'intuicyjnie', 'zrozumieć', 'wymyślić', 'kilkanaście', 'rok', 'trzymać', 'chcieć', 'popaść', 'lingwistyczny', 'definicyjny', 'dywagacja', 'oczywiście', 'świetnie', 'nadawać', 'kolejny', 'tekst', 'coś', 'inny']


## Ekstracja ważnych danych
#### Ekstrakcja Imienia i nazwiska

In [86]:
def extract_full_name(nlp_doc):
    pattern = [{'POS': 'PROPN '}, {'POS': 'PROPN'}]
    matcher.add('FULL_NAME', None, pattern)
    matches = matcher(nlp_doc)
    for match_id, start, end in matches:
        span = nlp_doc[start:end]
        return span.text

newText="Rafał Kostrzyński znajduje się teraz w Berlinie"
newTextDoc=nlp(newText)
matcher = Matcher(nlp.vocab)
print(extract_full_name(newTextDoc))

None


#### Ekstrakcja numeru

In [87]:
numberText=("Nie jestem pewny dlaczego nie działa lecz chciałbym ci podać swój numer telefonu (123) 456-789 ")

def extract_phone_number(nlp_doc):
    pattern = [{'ORTH': '('}, {'SHAPE': 'ddd'},
                    {'ORTH': ')'}, {'SHAPE': 'ddd'},
                     {'ORTH': '-', 'OP': '?'},
                     {'SHAPE': 'ddd'}]
    matcher.add('PHONE_NUMBER', None, pattern)
    matches = matcher(nlp_doc)
    for match_id, start, end in matches:
        span = nlp_doc[start:end]
        return span.text

conference_org_doc = nlp(numberText)
extract_phone_number(conference_org_doc)

'(123) 456-789'

## Ekstrakcja właściwości zależnie od gramatycznej struktury zdania

#### Utworzona zostanie struktura drzewna gdzie czasownik pełni funkcję korzenia

In [88]:
teaching_text = 'Rafal się uczy recytacji'
teaching_doc = nlp(teaching_text)
for token in teaching_doc:
    print(token.text, token.tag_,token.head.text, token.dep_)
#displacy.serve(teaching_doc, style='dep')

Rafal SUBST uczy nsubj
się QUB uczy expl:pv
uczy FIN uczy ROOT
recytacji SUBST uczy obj


## Nawigacja drzew oraz poddrzew

In [89]:
one_line_about_text = ('Rafał Kostrzyński programuje w javie oraz pracuje we francuskiej firmie')
one_line_about_doc = nlp(one_line_about_text)

#### Ekstrakcja dzieci od "programuje"

In [90]:
print([token.text for token in one_line_about_doc[2].children])

['Rafał', 'javie', 'pracuje']


#### Ekstrakcja poprzedniego, sąsiedniego węzłą

In [91]:
print (one_line_about_doc[2].nbor(-1))

Kostrzyński


#### Ekstrakcja sąsiedniego węzłą

In [92]:
print (one_line_about_doc[2].nbor())

w


#### Ekstrakcja wszystkich tokenów na lewo od "programuje"

In [93]:
print([token.text for token in one_line_about_doc[2].lefts])

['Rafał']


#### Ekstrakcja wszystkich tokenów na prawo od "programuje"

In [94]:
print([token.text for token in one_line_about_doc[2].rights])

['javie', 'pracuje']


#### Wyświetlenie poddrzewa

In [95]:
print (list(one_line_about_doc[2].subtree))

[Rafał, Kostrzyński, programuje, w, javie, oraz, pracuje, we, francuskiej, firmie]


#### Metoda tworząca zdanie z poddrzewa

In [96]:
def flatten_tree(tree):
    return ''.join([token.text_with_ws for token in list(tree)]).strip()
print (flatten_tree(one_line_about_doc[2].subtree))

Rafał Kostrzyński programuje w javie oraz pracuje we francuskiej firmie


## Ekstrakcja zdań z nieustrukturyzowanego tekstu
#### Detekcja zdań z fazą rzeczownika

In [97]:
conference_text =('Odbyje się konferencja deweloperska dnia 21. czerwca 2020 w Londynie')
conference_doc = nlp(conference_text)
for chunk in conference_doc.noun_chunks:
    print (chunk)

#### Detekcja fazy czasownika

In [98]:
about_talk_text=('Ta rozmowa przedstawi czytelnikowi informacje na temat użycia naturalnego przetwarzania języka w Fintech')
pattern = r'(<VERB>?<ADV>*<VERB>+)'
about_talk_doc = textacy.make_spacy_doc(about_talk_text,lang='pl_core_news_sm')
verb_phrases = textacy.extract.pos_regex_matches(about_talk_doc, pattern)
for chunk in verb_phrases:
    print(chunk.text)

for chunk in about_talk_doc.noun_chunks:
    print (chunk)

przedstawi


C:\Users\Rafal Kostrzynski\AppData\Local\Programs\Python\Python37\lib\site-packages\textacy\extract.py:338: DeprecationWarning: `pos_regex_matches()` has been deprecated! for similar but more powerful and performant functionality, use `textacy.extract.matches()` instead.
  action="once",


## Detekcja Encji nazwy

In [99]:
piano_class_text=('Akademia muzyczna znajduje się w Gdyni lub Londynie i ma najlepszych instruktorów gry na pianinie')
piano_class_doc = nlp(piano_class_text)
for ent in piano_class_doc.ents:
    print(ent.text, ent.start_char, ent.end_char,ent.label_, spacy.explain(ent.label_))

Akademia 0 8 orgName None
Gdyni 33 38 placeName None
Londynie 43 51 placeName None


#### Informacje na temat encji

In [100]:
#displacy.serve(piano_class_doc, style='ent')

#### Użycie NER do ukrycia osobistych informacji

In [101]:
survey_text =('Okazuje się, że Rafał Kostrzyński, Julie Fuller oraz Benjamin Brooks lubią jabłka. Kelly Cox i Matthew Evans lubią pomarańcze')
def replace_person_names(token):
    if token.ent_iob != 0 and token.ent_type_ == 'PERSON':
        return '[REDACTED] '
    return token.string

def redact_names(nlp_doc):
    for ent in nlp_doc.ents:
        ent.merge()
    tokens = map(replace_person_names, nlp_doc)
    return ''.join(tokens)

survey_doc = nlp(survey_text)
redact_names(survey_doc)





'Okazuje się, że Rafał Kostrzyński, Julie Fuller oraz Benjamin Brooks lubią jabłka. Kelly Cox i Matthew Evans lubią pomarańcze'